In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.


In [ ]:
train = pd.read_csv("../input/santander-value-prediction-challenge/train.csv")
test = pd.read_csv("../input/santander-value-prediction-challenge/test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [ ]:
#ordered_cols = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512','aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e','3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3','5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55','a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5','b85fa8b27','2155f5e16']

#ordered_ids = ['a70090dc7','5b74875d4','ca6c4ea0d','242407fc6','7ff66b22b','f43a48254','c9573f4ac','c14136677','08f544828','e77e32574','61636fa41','4c0316d32','414871b56','2c518cd87','9eb93baef','d63b2307d','db9ddeb12','7e45f5bd5','680ecfac8','814f58e7b','eb850ef06','6edaf114d']

#ordered_indexes = []
#for oi in ordered_ids:
    #ordered_indexes.append(train[train.ID == oi].index[0])

#tmp = train.loc[ordered_indexes, ["ID","target"] + ordered_cols]
#print('shape', tmp.shape)
#tmp

In [ ]:
#tmp.drop(['ID', 'target'], axis=1, inplace=True)

In [ ]:
#tmp.to_csv("new_pattern16.csv", index=False)

In [ ]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', 
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212',  '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', 
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a',
       '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2',  '0572565c2',
       '190db8488',  'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']

In [ ]:
cols1 = ['4b6dfc880', '266525925', '2cff4bf0c', 'a3382e205', '6488c8200', '547d3135b', 'b46191036', '453128993', '2599a7eb7', '2fc60d4d9', '009319104', 'de14e7687', 'aa31dd768', '2b54cddfd', 'a67d02050', '37aab1168', '939cc02f5', '31f72667c', '6f951302c', '54723be01', '4681de4fd', '8bd53906a', '435f27009', 'f82167572', 'd428161d9', '9015ac21d', 'ec4dc7883', '22c7b00ef', 'd4cc42c3d', '1351bf96e', '1e8801477', 'b7d59d3b5', 'a459b5f7d', '580f5ff06', '39b3c553a', '1eec37deb', '692c44993', 'ce8ce671e', '88ef1d9a8', 'bf042d928']
cols2 = ['b26d16167', '930f989bf', 'ca58e6370', 'aebe1ea16', '03c589fd7', '600ea672f', '9509f66b0', '70f4f1129', 'b0095ae64', '1c62e29a7', '32a0342e2', '2fc5bfa65', '026ca57fd', '49e68fdb9', '09c81e679', 'aacffd2f4', '61483a9da', '29725e10e', '227ff4085', '5878b703c', '50a0d7f71', '0d1af7370', '7c1af7bbb', '4bf056f35', '3dd64f4c4', 'b9f75e4aa', '423058dba', '150dc0956', 'adf119b9a', 'a8110109e', '6c4f594e0', 'c44348d76', 'db027dbaf', '1fcba48d0', '6ff9b1760', '8d12d44e1', '8d13d891d', '482715cbd', 'f81c2f1dd', 'dda820122']
cols3 = ['81de0d45e', '543c24e33', '18562fc62', '0256b6714', 'd6006ff44', 'e3a38370e', '6a323434b', '7c444370b', '8d2d050a2', '9657e51e1', '13f3a3d19', 'b5c839236', '70f3033c6', '849125d91', 'f4b374613', '16b532cdc', '88219c257', 'fd1102929', '74fb8f14c', '699712087', '22501b58e', '2c42b0dce', '9e9274b24', '5263c204d', '526ed2bec', '2c95e6e31', '01f7de15d', 'cdbe394fb', 'd0f65188c', 'ef1e1fac8', 'adf357c9b', 'b8a716ebf', 'a3f2345bf', '110e4132e', '680159bab', '586b23138', 'af91c41f0', 'f1a1562cd', '9f2f1099b', 'bf0e69e55']
cols4 = ['bc70cbc26', 'aca228668', 'a6e871369', '0dc4d6c7d', '2155f5e16', '070f95c99', '7e814a30d', '939f628a7', 'b85fa8b27', '794e93ca6', 'ef30f6be5', 'ea4046b8d', '4da206d28', '13ee58af1', 'fb387ea33', 'a8b721722', 'f39074b55', '1184df5c2', '2288333b4', '5d3b81ef8', 'e222309b0', '22ed6dba3', '5547d6e11', '5b943716b', 'fec5eaf1a', 'a396ceeb9', '9280f3d04', 'a00adf70e', '2b2a10857', '3a48a2cd2', '8f3740670', 'cd24eae8a', '342e7eb03', '73687e512', 'aa164b93b', 'f1e0ada11', '83c3779bf', 'edc84139a', '9df4daa99', 'ced6a7e91']
cols4.reverse()
cols5 = ['090dfb7e2', '48282f315', '1bf2dfd4a', '50603ae3d', '6ccaaf2d7', '50b1dd40f', '1604c0735', 'e94c03517', 'f9378f7ef', '65266ad22', 'ac61229b6', '1ced7f0b4', 'f5723deba', 'b9a4f06cd', 'df28ac53d', '8132d18b8', 'ae825156f', '936dc3bc4', '5b233cf72', '95a2e29fc', '882a3da34', '2cb4d123e', 'c83d6b24d', '90a2428a5', '0e1921717', '67e6c62b9', '900045349', '320931ca8', 'bf89fac56', 'da3b0b5bb', 'f06078487', '56896bb36', 'a79522786', '71c2f04c9', '4b1a994cc', '1af96abeb', 'dee843499', '645b47cde', 'cc9c2fc87', 'a8e15505d']
cols6 = ['166008929', 'f02ecb19c', '861076e21', 'a09a238d0', '935ca66a9', '9de83dc23', 'f97d9431e', '77deffdf0', 'bb0ce54e9', '11e12dbe8', 'c0d2348b7', 'ca2b906e8', 'c8d582dd2', 'bd6da0cca', '8781e4b91', 'cbb673163', 'c10f31664', '62fb56487', '68a945b18', 'ad009c8b9', 'ea772e115', '4bcf15776', '1fe5d56b9', '51d4053c7', '44d5b820f', 'b4cfe861f', 'b7c931383', '122c135ed', '6c5c8869c', 'dfd179071', 'ce3d7595b', '0d51722ca', 'f6eba969e', 'a6b6bc34a', 'e078302ef', 'd5fa73ead', '1c71183bb', '63c094ba4', '2e103d632', '87ffda550']
cols6.reverse()
cols7 = ['c928b4b74', '8e4d0fe45', '6c0e0801a', '02861e414', '041c5d0c9', 'aac52d8d9', 'e7c0cfd0f', 'd48c08bda', 'd7875bb6c', '0c9462c08', '57dd44c29', 'a93118262', '850027e38', 'db3839ab0', '27461b158', '32174174c', '9306da53f', '95742c2bf', '5831f4c76', '1e6306c7c', '06393096a', '9a07d7b1f', 'd7d314edc', '13bdd610a', '4d2671746', '822e49b95', '3c8a3ced0', '83635fb67', '1857fbccf', 'b6c0969a2', 'c4972742d', '36a9a8479', '092271eb3', '74d7f2dc3', 'e78e3031b', 'e79e5f72c', '277ef93fc', 'b30e932ba', '350473311', '8f57141ec']
cols8 = ['316b978cd', 'a8ee55662', '2ab5a56f5', '62a915028', '21af91e9b', '73e591019', '363713112', '0c545307d', '15e8a9331', '2a71f4027', 'f118f693a', '1a7de209c', 'b2790ef54', '97cc1b416', 'a79b1f060', '3519bf4a4', '42088cf50', 'ddef5ad30', '0872fe14d', 'fdd07cac1', '1bf37b3e2', 'c110ee2b7', '64bf3a12a', '5747a79a9', '6a0b386ac', 'd9fc63fa1', 'f3ee6ba3c', '6f113540d', '312832f30', '2e0148f29', '39e1796ab', '051410e3d', '7950f4c11', '0231f07ed', 'd83da5921', 'faf7285a1', 'ed1d5d137', '497adaff8', '62ffce458', 'f1eeb56ae']
cols8.reverse()
cols9 = ['48b839509', '2b8851e90', '28f75e1a5', '0e3ef9e8f', '7ca10e94b', '37ac53919', '467aa29ce', '4b6c549b1', '74c5d55dc', '44f3640e4', '0700acbe1', 'e431708ff', 'a0453715a', 'd1fd0b9c2', '097836097', '9e3aea49a', '899dbe405', '525635722', '87a2d8324', 'd421e03fd', 'faf024fa9', '1254b628a', 'a19b05919', '34a4338bc', '08e89cc54', 'a29c9f491', '62ea662e7', 'a0a8005ca', '5fe6867a4', '8b710e161', '7ab926448', 'ff2c9aa8f', 'd04e16aed', 'b625fe55a', '4e5da0e96', '7124d86d9', 'b6fa5a5fd', '215c4d496', '55a7e0643', '0a26a3cfe']
cols10 = ['df69cf626', '43782ef36', '438b8b599', 'b0fcfeab8', 'e1b20c3a6', 'ac7a97382', 'd5efae759', '04ecdcbb3', 'ade8a5a19', '54f26ee08', '5ed0c24d0', '8722f33bb', '88c0ec0a6', '5cc9b6615', '3d655b0ed', '18b4fa3f5', '4af2493b6', 'ec5fb550f', '71d64e3f7', 'a65b73c87', 'cb5161856', 'cd15bb515', '303572ae2', '31015eaab', 'cfd255ee3', '2a9fed806', 'feb40ad9f', 'ce53d1a35', 'a8f80f111', 'c283d4609', 'e8bd579ae', '590b24ab1', '7298ca1ef', '145c7b018', 'c5dacc85b', '91570fb11', '78c57d7cd', '15ea45005', '93efdb50f', '920a04ee2']
cols10.reverse()
cols11 = ['02827212f', '6f53aee73', 'd45fd5508', 'e97fa47e4', 'e32ad270b', 'b66bf9d44', '7194699cd', 'c30399758', 'c436c7e73', '1c4157dfd', 'a2a1975d6', 'd80abf8bc', 'b74ef4294', 'e3846e931', '6723b1708', '76e9423c3', '6931ed626', 'a636266f3', '7f9e0d947', 'dba14a5d4', '5d60b9ba7', '3fa6c395f', 'bbd16b7a0', '4c48708d8', '73a8a4d75', 'ea72c62a1', '60cb16e88', '851697562', 'ecbd077d0', '03a4ccd7c', '5adfe7419', '20e2c484e', '6bee3733e', '74f3ac6af', 'de7063efa', '9397535c7', '0ccd6454a', '8984e4066']
cols11.reverse()
cols12 = ['34d3715d5', 'd0d340214', '9c404d218', 'a1b169a3a', 'c624e6627', 'c144a70b1', 'b36a21d49', 'dfcf7c0fa', 'c63b4a070', '43ebb15de', '1f2a670dd', '3f07a4581', '0b1560062', 'e9f588de5', '9ed0e6ddb', '65d14abf0', '0b790ba3a', '9e89978e3', 'ee6264d2b', 'c86c0565e', '4de164057', '87ba924b1', '4d05e2995', '2c0babb55', 'e9375ad86', '8988e8da5', '8a1b76aaf', '654dd8a3b', '724b993fd', 'f423cf205', '3b54cc2cf', 'e04141e42', 'cacc1edae', '2c339d4f2', '314396b31', '3f8614071', '16d1d6204', '80b6e9a8b', 'a84cbdab5', '1a6d13c4a']
cols13 = ['6cf7644e0', 'c0004231c', '25968f96e', 'fb8395d97', 'ee9e70298', 'eda856f5f', 'e662f1672', '49f11e712', '43dc0f90c', '11d9e8383', '0738fe5ff', '3268914c7', '08d17e384', '1a67a2101', '406a11b5a', 'e2e6f1d23', '2862eec4f', '9c6bf2983', '0a4e510ee', '14d2c6d95', '0a03426de', '82ade3db6', '8909b68e1', 'aaed60a3d', '879e1f51a', '5cbf1b3ea', '1ecddbaa0', 'b96718230', 'bd550871c', 'c55eb4392', '0743b9c08', '5580c77b0', 'de66047b0', '25613ca0a', '2322dbbbb', '760ef38f1', 'da5c36eee', 'a760e7f65', '2de811f82', '5d8a2a27d']
cols14 = ['14c2463ff', '14a5969a6', 'dc07f7e11', '2b85882ad', 'e5a8e9154', '786351d97', 'b2e1308ae', 'b05eae352', '3c209d9b6', 'bf8150471', '542f770e5', 'd4e8dd865', '9581ec522', '4ceef6dbd', '169f6dda5', 'bc2bf3bcd', '2bbcbf526', 'd7978c11c', '625525b5d', 'f0aa40974', '35da68abb', '5f5cfc3c0', '9b39b02c0', 'c87f4fbfb', '6bab7997a', '21e0e6ae3', '3a90540ab', 'dc60842fb', 'df6a71cc7', 'ede70bfea', '07cb6041d', 'c5cb7200e', 'e5ddadc85', 'b14026520', 'dcfcddf16', 'b25319cb3', '3be4dad48', '3d23e8abd', '9fa984817', '1b681c3f0']
cols14.reverse()
cols15 = ['994b946ad', 'acc4a8e68', '4ecc3f505', '64dd02e44', '8479174c2', '6192f193d', 'ba136ae3f', '96b6bd42b', '95aea9233', '44cb9b7c4', '578b81a77', '7121c40ee', '018ab6a80', 'f9847e9fe', '578eda8e0', '097c7841e', 'e43343256', 'ddea5dc65', '415094079', '2570e2ba9', 'c671db79e', 'd79736965', '429687d5a', 'e4159c59e', '64534cc93', 'aeff360c7', '3a62b36bd', 'c8fdf5cbf', '0e1f6696a', 'f14b57b8f', 'cf488d633', '6d2ece683', '9c42bff81', 'ba4ceabc5', 'd4c1de0e2', 'ac30af84a', 'ea4887e6b', '197cb48af', 'e1d0e11b5', '9d5c7cb94']
cols15.reverse()
cols16 = ['c2dae3a5a', '7623d805a', '540cc3cd1', 'f8cd9ae02', 'de4e75360', '230a025ca', '4b9540ab3', '371da7669', 'ed8951a75', 'b8f892930', '6f88afe65', 'f16a196c6', '3f4a39818', 'd6470c4ce', '66146c12d', 'b98f3e0d7', '84d9d1228', '251d1aa17', 'a39758dae', '45f6d00da', '5ac278422', '6cf7866c1', 'dc10234ae', '23db7d793', 'bdf773176', '469630e5c', 'dcc269cfe', '72d34a148', '6fa35fbba', 'c7525612c', 'fb5f5836e', '30b3daec2', '0a953f97e', '29c059dd2', '961b91fe7', 'cfc1ce276', '5a1589f1a', '3d6d38290', '3bdee45be', 'b6daeae32']
cols16.reverse()
cols17 = ['f3cf9341c', 'fa11da6df', 'd47c58fe2', '0d5215715', '555f18bd3', '134ac90df', '716e7d74d', 'c00611668', '1bf8c2597', '1f6b2bafa', '174edf08a', '5bc7ab64f', 'f1851d155', 'a61aa00b0', 'b2e82c050', '26417dec4', '53a550111', 'e8d9394a0', '51707c671', 'cbbc9c431', '6b119d8ce', 'f296082ec', 'be2e15279', '698d05d29', '38e6f8d32', '93ca30057', '7af000ac2', '1fd0a1f2a', '41bc25fef', '0df1d7b9a', '88d29cfaf', '2b2b5187e', 'bf59c51c3', 'cfe749e26', 'ad207f7bb', '11114a47a', 'a8dd5cea5', '341daa7d1', '7b672b310', 'b88e5de84']
cols18 = ['593cccdab', '99cc87fd7', '4c53b206e', '26628e8d8', '1de4d7d62', '5780e6ffa', 'afb6b8217', 'e9a8d043d', '3cea34020', '68c7cf320', '8d8bffbae', 'ac0493670', '5c220a143', 'e26299c3a', '0b1741a7f', 'a682ef110', '9886b4d22', '59dfc16da', '94072d7a3', '1e16f11f3', '22c019a2e', '89ebc1b76', '829fb34b8', 'c24ea6548', 'c906cd268', '07746dcda', '45319105a', 'fcda960ae', '56f619761', '6b241d083', 'c6850e7db', '1834f29f5', '9d5af277d', 'fc436be29', '2165c4b94', '0616154cc', '5244415dd', '6eaea198c', '9126049d8', 'a3e023f65']
cols18.reverse()
cols19 = ['f0eb7b98f', 'bb6f50464', '988518e2d', 'd7447b2c5', 'd53d64307', 'e700276a2', '6a542a40a', 'fe28836c3', '1f8415d03', '9a1d8054b', '0c9b00a91', '800f38b6b', 'ff3b49c1d', 'e4e9c8cc6', 'b77c707ef', 'b22eb2036', '5807de036', '3a7771f56', 'da7f4b066', 'ccf6632e6', 'd746efbfe', 'f1c272f04', 'e36687647', 'c059f2574', 'd58172aef', 'de63b3487', '4c835bd02', 'fe0dd1a15', 'b3abb64d2', '5a6a1ec1a', '33e4f9a0e', '946d16369', '56cb93fd8', '854e37761', '1f71b76c1', '1029d9146', '3a89d003b', '759c9e85d', 'a9819bda9', 'ea26c7fe6']
cols19.reverse()
cols20 = ['e613715cc', '375c6080e', '3c4df440f', 'cef9ab060', '93521d470', '46c525541', 'a31ba11e6', '64cabb6e7', '85ef8a837', 'f99a09543', 'c83fc48f2', 'd6af4ee1a', 'cde3e280a', '337b3e53b', '1d0aaa90f', '22abeffb6', 'c46028c0f', 'a5c62af4a', 'feeb05b3f', '45293f374', '8c61bede6', '7acae7ae9', '13d6a844f', '168c50797', 'a240f6da7', '00f844fea', 'c4de134af', '65aa7f194', '6654ce6d8', 'f6bdb908a', 'eb4a20186', '5ee81cb6e', '4e21c4881', '823ac378c', '304ebcdbc', '4c60b70b8', 'ceba761ec', '39549da61', '9d4428628', '37f11de5d']
cols20.reverse()
cols21 = ['ccd9fc164', 'd7f15a3ad', 'e585cbf20', '03055cc36', 'f04f0582d', '1791b43b0', 'bca395b73', 'b02dfb243', 'c901e7df1', 'f5e0f4a16', '5edd220bc', '20a325694', 'd93058147', 'c91a4f722', '9a5cd5171', 'b5df42e10', 'b1c5346c4', '177993dc6', '07df9f30c', '71f5ab59f', 'ee7fb1067', '761b8e0ec', '42ed6824a', '77fa93749', '3c9db4778', '83e3e2e60', '9f5f58e61', '3fa0b1c53', 'e769ee40d', '60a5b79e4', '518b5da24', '632fed345', 'cfd55f2b6', '74a7b9e4a', 'f8a437c00', '1010d7174', 'bb0408d98', 'a9ca6c2f4', '4ec3bfda8', '06148867b']
cols21.reverse()
cols22 = ['a1cd7b681', '9b490abb3', 'b10f15193', '05f54f417', 'a7ac690a8', 'ed6c300c2', 'b1bb8eac3', 'd0803e3a1', 'bd1c19973', '84ec1e3db', 'a34f8d443', '24018f832', '82e01a220', '4c2064b00', '0397f7c9b', 'ba42e41fa', '22d7ad48d', 'dbfa2b77f', '9abffd22c', '2c6c62b54', '9fa38def3', 'ecb354edf', '9c3154ae6', '2f26d70f4', '53102b93f', 'a36b95f78', '1fa0f78d0', '19915a6d3', 'c944a48b5', '482b04cba', '2ce77a58f', 'c3f400e36', '86558e595', '20305585c', 'f8ccfa064', 'dd771cb8e', '9aa27017e', 'cd7f0affd', '236cc1ff5', 'a3fc511cd']
cols23 = ['8337d1adc', '8675bec0b', '1d04efde3', '2e3c96323', 'bee629024', '5e645a169', '1977eaf08', '191e21b5f', 'b64425521', 'a4511cb0b', '949ed0965', '6b795a2bc', '956d228b9', '64c6eb1cb', '8618bc1fd', 'a3fb07bfd', '467bee277', '5324862e4', '031490e77', 'cdfc2b069', '6df033973', '2eeadde2b', '99e779ee0', '9f7b782ac', '1dd7bca9f', 'e16a20511', '04b88be38', 'a3ef69ad5', 'f9e3b03b7', 'd1e0f571b', '37f57824c', '796c218e8', 'b9ba17eb6', 'e13b0c0aa', 'a9f61cf27', '1af4d24fa', '61c1b7eb6', '2cb73ede7', '4dcf81d65', 'df838756c']
cols23.reverse()

In [ ]:
rows = ['19bafc0eb','8ef819e43','5342778cc','96b1c70cf','359d25b38','c71842c11',
        'ccfb14bf1','9d9176f62','c989a1ff0','7d4b5b291','102bd04d8','9eb312e66',
        '557b8a17e','eb13cb6a1','fa5455b9c','1a2fcf1b6','1a4fdc864','1a060c889',
        'c114630ea','3bf4d0dd2','87daaf67b','7e8d06e00','0dadd8851','e8b380c2a',
        '12b150758','9db0f2cd6','802f25a23','59f52b75a','1e49872f3','4fa3167f8',
        'bb5041e89','f2a11f83b','f84109ff1','47c4e8ec2','6b889a750','3540b1ba4',
        '71286b738','de5f122d8','fa93b66c1','e1fffe216','693802083','8e8c910ce',
        '3bd8c8e64','319e95726','5ea3ca394','720907682','d08699be7','97ebb131c',
        '499520a32','d49a3bd3e','571d2ea50','a74e200d2','bf99827e7','e6139f9c2',
        '2e1ce6102','8eb4c28c7','96630c88a','15defc71c','d0a52a9a5','917a29ba4',
        'eec39f669','4eb3978dd','0110c05db','f11ffbbaf','36153f507','86f59f6c9',
        '18516f2bb','a1f713030','646e59743','e677c32f6','52bdaa610','17224efee',
        '2d6fff5cc','7ec5a9a38','376d7dced','22cd3d13a','688b43e3e','f637c0f24',
        '810285ac0','1653b322d','ef3f27c0c','ef111073a','3499dd5b0','987541b9c',
        '6cf4b79b1','567a57827','3f5a7464e','d339b8d29','0f2e84a29','b4e4afdf5',
        '815281bbe','96e9102e5','e375d6c21','08d27660b','67c967a26','3fb98b688',
        'b82c7eef3','a37b27d20','b688e11af','cb7691896','7ba151fb4']

In [ ]:
test["target"] = train["target"].mean()
all_df = pd.concat([train, test]).reset_index(drop=True)
all_df.columns = all_df.columns.astype(str)
print(all_df.shape)

In [ ]:
train.index = train.ID
a = train.loc[rows, cols1]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern17.csv', index=False)

In [ ]:
#train.index = train.ID
a = train.loc[rows, cols2]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern18.csv', index=False)

In [ ]:
a = train.loc[rows, cols3]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern19.csv', index=False)

In [ ]:
a = train.loc[rows, cols4]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern20.csv', index=False)

In [ ]:
a = train.loc[rows, cols5]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern21.csv', index=False)

In [ ]:
a = train.loc[rows, cols6]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern22.csv', index=False)

In [ ]:
a = train.loc[rows, cols7]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern23.csv', index=False)

In [ ]:
a = train.loc[rows, cols8]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern24.csv', index=False)

In [ ]:
a = train.loc[rows, cols9]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern25.csv', index=False)

In [ ]:
a = train.loc[rows, cols10]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern26.csv', index=False)

In [ ]:
a = train.loc[rows, cols11]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern27.csv', index=False)

In [ ]:
a = train.loc[rows, cols12]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern28.csv', index=False)

In [ ]:
a = train.loc[rows, cols13]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern29.csv', index=False)

In [ ]:
a = train.loc[rows, cols14]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern30.csv', index=False)

In [ ]:
a = train.loc[rows, cols15]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern31.csv', index=False)

In [ ]:
a = train.loc[rows, cols16]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern32.csv', index=False)

In [ ]:
a = train.loc[rows, cols17]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern33.csv', index=False)

In [ ]:
a = train.loc[rows, cols18]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern34.csv', index=False)

In [ ]:
a = train.loc[rows, cols19]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern35.csv', index=False)

In [ ]:
a = train.loc[rows, cols20]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern36.csv', index=False)

In [ ]:
a = train.loc[rows, cols21]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern37.csv', index=False)

In [ ]:
a = train.loc[rows, cols22]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern38.csv', index=False)

In [ ]:
a = train.loc[rows, cols23]
a = a.reset_index()
a.drop('ID', axis=1, inplace=True)
a.to_csv('new_pattern39.csv', index=False)

In [ ]:
colgroups = [
    ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'],
    ['266525925', '4b6dfc880', '2cff4bf0c', 'a3382e205', '6488c8200', '547d3135b', 'b46191036', '453128993', '2599a7eb7', '2fc60d4d9', '009319104', 'de14e7687', 'aa31dd768', '2b54cddfd', 'a67d02050', '37aab1168', '939cc02f5', '31f72667c', '6f951302c', '54723be01', '4681de4fd', '8bd53906a', '435f27009', 'f82167572', 'd428161d9', '9015ac21d', 'ec4dc7883', '22c7b00ef', 'd4cc42c3d', '1351bf96e', '1e8801477', 'b7d59d3b5', 'a459b5f7d', '580f5ff06', '39b3c553a', '1eec37deb', '692c44993', 'ce8ce671e', '88ef1d9a8', 'bf042d928'],
    ['9d5c7cb94', '197cb48af', 'ea4887e6b', 'e1d0e11b5', 'ac30af84a', 'ba4ceabc5', 'd4c1de0e2', '6d2ece683', '9c42bff81', 'cf488d633', '0e1f6696a', 'c8fdf5cbf', 'f14b57b8f', '3a62b36bd', 'aeff360c7', '64534cc93', 'e4159c59e', '429687d5a', 'c671db79e', 'd79736965', '2570e2ba9', '415094079', 'ddea5dc65', 'e43343256', '578eda8e0', 'f9847e9fe', '097c7841e', '018ab6a80', '95aea9233', '7121c40ee', '578b81a77', '96b6bd42b', '44cb9b7c4', '6192f193d', 'ba136ae3f', '8479174c2', '64dd02e44', '4ecc3f505', 'acc4a8e68', '994b946ad'],
    ['f1eeb56ae', '62ffce458', '497adaff8', 'ed1d5d137', 'faf7285a1', 'd83da5921', '0231f07ed', '7950f4c11', '051410e3d', '39e1796ab', '2e0148f29', '312832f30', '6f113540d', 'f3ee6ba3c', 'd9fc63fa1', '6a0b386ac', '5747a79a9', '64bf3a12a', 'c110ee2b7', '1bf37b3e2', 'fdd07cac1', '0872fe14d', 'ddef5ad30', '42088cf50', '3519bf4a4', 'a79b1f060', '97cc1b416', 'b2790ef54', '1a7de209c', '2a71f4027', 'f118f693a', '15e8a9331', '0c545307d', '363713112', '73e591019', '21af91e9b', '62a915028', '2ab5a56f5', 'a8ee55662', '316b978cd'],
    ['b26d16167', '930f989bf', 'ca58e6370', 'aebe1ea16', '03c589fd7', '600ea672f', '9509f66b0', '70f4f1129', 'b0095ae64', '1c62e29a7', '32a0342e2', '2fc5bfa65', '09c81e679', '49e68fdb9', '026ca57fd', 'aacffd2f4', '61483a9da', '227ff4085', '29725e10e', '5878b703c', '50a0d7f71', '0d1af7370', '7c1af7bbb', '4bf056f35', '3dd64f4c4', 'b9f75e4aa', '423058dba', '150dc0956', 'adf119b9a', 'a8110109e', '6c4f594e0', 'c44348d76', 'db027dbaf', '1fcba48d0', '8d12d44e1', '8d13d891d', '6ff9b1760', '482715cbd', 'f81c2f1dd', 'dda820122'],
    ['c928b4b74', '8e4d0fe45', '6c0e0801a', '02861e414', 'aac52d8d9', '041c5d0c9', 'd7875bb6c', 'e7c0cfd0f', 'd48c08bda', '0c9462c08', '57dd44c29', 'a93118262', '850027e38', 'db3839ab0', '27461b158', '32174174c', '9306da53f', '95742c2bf', '5831f4c76', '1e6306c7c', '06393096a', '13bdd610a', 'd7d314edc', '9a07d7b1f', '4d2671746', '822e49b95', '3c8a3ced0', '83635fb67', '1857fbccf', 'c4972742d', 'b6c0969a2', 'e78e3031b', '36a9a8479', 'e79e5f72c', '092271eb3', '74d7f2dc3', '277ef93fc', 'b30e932ba', '8f57141ec', '350473311'],
    ['06148867b', '4ec3bfda8', 'a9ca6c2f4', 'bb0408d98', '1010d7174', 'f8a437c00', '74a7b9e4a', 'cfd55f2b6', '632fed345', '518b5da24', '60a5b79e4', '3fa0b1c53', 'e769ee40d', '9f5f58e61', '83e3e2e60', '77fa93749', '3c9db4778', '42ed6824a', '761b8e0ec', 'ee7fb1067', '71f5ab59f', '177993dc6', '07df9f30c', 'b1c5346c4', '9a5cd5171', 'b5df42e10', 'c91a4f722', 'd93058147', '20a325694', 'f5e0f4a16', '5edd220bc', 'c901e7df1', 'b02dfb243', 'bca395b73', '1791b43b0', 'f04f0582d', 'e585cbf20', '03055cc36', 'd7f15a3ad', 'ccd9fc164'],
    ['df838756c', '2cb73ede7', '4dcf81d65', '61c1b7eb6', 'a9f61cf27', '1af4d24fa', 'e13b0c0aa', 'b9ba17eb6', '796c218e8', '37f57824c', 'd1e0f571b', 'f9e3b03b7', 'a3ef69ad5', 'e16a20511', '04b88be38', '99e779ee0', '9f7b782ac', '1dd7bca9f', '2eeadde2b', '6df033973', 'cdfc2b069', '031490e77', '5324862e4', '467bee277', 'a3fb07bfd', '64c6eb1cb', '8618bc1fd', '6b795a2bc', '956d228b9', '949ed0965', 'a4511cb0b', 'b64425521', '2e3c96323', '191e21b5f', 'bee629024', '1977eaf08', '5e645a169', '1d04efde3', '8675bec0b', '8337d1adc'],
    ['a1cd7b681', '9b490abb3', 'b10f15193', '05f54f417', 'a7ac690a8', 'ed6c300c2', 'd0803e3a1', 'b1bb8eac3', 'bd1c19973', 'a34f8d443', '84ec1e3db', '24018f832', '82e01a220', '4c2064b00', '0397f7c9b', 'ba42e41fa', '22d7ad48d', '9abffd22c', 'dbfa2b77f', '2c6c62b54', '9fa38def3', 'ecb354edf', '9c3154ae6', '2f26d70f4', '53102b93f', 'a36b95f78', '1fa0f78d0', '19915a6d3', 'c944a48b5', '482b04cba', '2ce77a58f', '86558e595', 'c3f400e36', '20305585c', 'f8ccfa064', 'dd771cb8e', '9aa27017e', 'cd7f0affd', '236cc1ff5', 'a3fc511cd'],
    ['920a04ee2', '93efdb50f', '15ea45005', '78c57d7cd', '91570fb11', 'c5dacc85b', '145c7b018', '590b24ab1', 'c283d4609', 'e8bd579ae', '7298ca1ef', 'ce53d1a35', 'a8f80f111', '2a9fed806', 'feb40ad9f', 'cfd255ee3', '31015eaab', '303572ae2', 'cd15bb515', 'cb5161856', 'a65b73c87', '71d64e3f7', 'ec5fb550f', '4af2493b6', '18b4fa3f5', '3d655b0ed', '5cc9b6615', '88c0ec0a6', '8722f33bb', '5ed0c24d0', '54f26ee08', '04ecdcbb3', 'ade8a5a19', 'd5efae759', 'ac7a97382', 'e1b20c3a6', 'b0fcfeab8', '438b8b599', '43782ef36', 'df69cf626'],
    ['50603ae3d', '48282f315', '090dfb7e2', '6ccaaf2d7', '1bf2dfd4a', '50b1dd40f', '1604c0735', 'e94c03517', 'f9378f7ef', '65266ad22', 'ac61229b6', 'f5723deba', '1ced7f0b4', 'b9a4f06cd', '8132d18b8', 'df28ac53d', 'ae825156f', '936dc3bc4', '5b233cf72', '95a2e29fc', '882a3da34', '2cb4d123e', '0e1921717', 'c83d6b24d', '90a2428a5', '67e6c62b9', '320931ca8', '900045349', 'bf89fac56', 'da3b0b5bb', 'f06078487', '56896bb36', 'a79522786', '71c2f04c9', '1af96abeb', '4b1a994cc', 'dee843499', '645b47cde', 'a8e15505d', 'cc9c2fc87'],
    ['b6daeae32', '3bdee45be', '3d6d38290', '5a1589f1a', '961b91fe7', '29c059dd2', 'cfc1ce276', '0a953f97e', '30b3daec2', 'fb5f5836e', 'c7525612c', '6fa35fbba', '72d34a148', 'dcc269cfe', 'bdf773176', '469630e5c', '23db7d793', 'dc10234ae', '5ac278422', '6cf7866c1', 'a39758dae', '45f6d00da', '251d1aa17', '84d9d1228', 'b98f3e0d7', '66146c12d', 'd6470c4ce', '3f4a39818', 'f16a196c6', 'b8f892930', '6f88afe65', 'ed8951a75', '371da7669', '4b9540ab3', '230a025ca', 'f8cd9ae02', 'de4e75360', '540cc3cd1', '7623d805a', 'c2dae3a5a'],
    ['d0d340214', '34d3715d5', '9c404d218', 'c624e6627', 'a1b169a3a', 'c144a70b1', 'b36a21d49', 'dfcf7c0fa', 'c63b4a070', '43ebb15de', '1f2a670dd', '3f07a4581', '0b1560062', 'e9f588de5', '65d14abf0', '9ed0e6ddb', '0b790ba3a', '9e89978e3', 'ee6264d2b', 'c86c0565e', '4de164057', '87ba924b1', '4d05e2995', '2c0babb55', 'e9375ad86', '8988e8da5', '8a1b76aaf', '724b993fd', '654dd8a3b', 'f423cf205', '3b54cc2cf', 'e04141e42', 'cacc1edae', '314396b31', '2c339d4f2', '3f8614071', '16d1d6204', '80b6e9a8b', 'a84cbdab5', '1a6d13c4a'],
    ['a9819bda9', 'ea26c7fe6', '3a89d003b', '1029d9146', '759c9e85d', '1f71b76c1', '854e37761', '56cb93fd8', '946d16369', '33e4f9a0e', '5a6a1ec1a', '4c835bd02', 'b3abb64d2', 'fe0dd1a15', 'de63b3487', 'c059f2574', 'e36687647', 'd58172aef', 'd746efbfe', 'ccf6632e6', 'f1c272f04', 'da7f4b066', '3a7771f56', '5807de036', 'b22eb2036', 'b77c707ef', 'e4e9c8cc6', 'ff3b49c1d', '800f38b6b', '9a1d8054b', '0c9b00a91', 'fe28836c3', '1f8415d03', '6a542a40a', 'd53d64307', 'e700276a2', 'bb6f50464', '988518e2d', 'f0eb7b98f', 'd7447b2c5'],
    ['87ffda550', '63c094ba4', '2e103d632', '1c71183bb', 'd5fa73ead', 'e078302ef', 'a6b6bc34a', 'f6eba969e', '0d51722ca', 'ce3d7595b', '6c5c8869c', 'dfd179071', '122c135ed', 'b4cfe861f', 'b7c931383', '44d5b820f', '4bcf15776', '51d4053c7', '1fe5d56b9', 'ea772e115', 'ad009c8b9', '68a945b18', '62fb56487', 'c10f31664', 'cbb673163', 'c8d582dd2', '8781e4b91', 'bd6da0cca', 'ca2b906e8', '11e12dbe8', 'bb0ce54e9', 'c0d2348b7', '77deffdf0', 'f97d9431e', 'a09a238d0', '935ca66a9', '9de83dc23', '861076e21', 'f02ecb19c', '166008929'],
    ['f3cf9341c', 'fa11da6df', 'd47c58fe2', '0d5215715', '555f18bd3', '134ac90df', '716e7d74d', 'c00611668', '1bf8c2597', '1f6b2bafa', '174edf08a', 'f1851d155', '5bc7ab64f', 'a61aa00b0', 'b2e82c050', '26417dec4', '53a550111', '51707c671', 'e8d9394a0', 'cbbc9c431', '6b119d8ce', 'f296082ec', 'be2e15279', '698d05d29', '38e6f8d32', '93ca30057', '7af000ac2', '1fd0a1f2a', '41bc25fef', '0df1d7b9a', '88d29cfaf', '2b2b5187e', 'bf59c51c3', 'cfe749e26', 'ad207f7bb', '11114a47a', '341daa7d1', 'a8dd5cea5', '7b672b310', 'b88e5de84'],
]

In [ ]:
pattern_1964666 = pd.read_csv('../input/pattern/pattern-found/pattern_1964666.66.csv')
pattern_1166666 = pd.read_csv('../input/pattern/pattern-found/pattern_1166666.66.csv')
pattern_812666 = pd.read_csv('../input/pattern/pattern-found/pattern_812666.66.csv')
pattern_2002166 = pd.read_csv('../input/pattern/pattern-found/pattern_2002166.66.csv')
pattern_3160000 = pd.read_csv('../input/pattern/pattern-found/pattern_3160000.csv')
pattern_3255483 = pd.read_csv('../input/pattern/pattern-found/pattern_3255483.88.csv')
pattern_new = pd.read_csv("../input/newfound/new_pattern.csv")
pattern_new1 = pd.read_csv("../input/newfound1/new_pattern1.csv")
pattern_new2 = pd.read_csv("../input/newfound2/new_pattern2.csv")
pattern_new3 = pd.read_csv("../input/newfound2/new_pattern3.csv")
pattern_new4 = pd.read_csv("../input/newfound2/new_pattern4.csv")
pattern_new5 = pd.read_csv("../input/newfound2/new_pattern5.csv")
#pattern_new6 = pd.read_csv("../input/newfound2/new_pattern6.csv")
pattern_new7 = pd.read_csv("../input/newfound2/new_pattern7.csv")
pattern_new8 = pd.read_csv("../input/newfound2/new_pattern8.csv")
pattern_new9 = pd.read_csv("../input/newfound2/new_pattern9.csv")
pattern_new10 = pd.read_csv("../input/newfound2/new_pattern10.csv")
pattern_new11 = pd.read_csv("../input/newfound2/new_pattern11.csv")
pattern_new12 = pd.read_csv("../input/newfound2/new_pattern12.csv")
pattern_new13 = pd.read_csv("../input/newfound2/new_pattern13.csv")
pattern_new14 = pd.read_csv("../input/newfound2/new_pattern14.csv")
pattern_new15 = pd.read_csv("../input/newfound2/new_pattern15.csv")
pattern_new16 = pd.read_csv("../input/newfound3/new_pattern16.csv")
pattern_new17 = pd.read_csv("../input/newfound3/new_pattern17.csv")
pattern_new18 = pd.read_csv("../input/newfound3/new_pattern18.csv")
pattern_new19 = pd.read_csv("../input/newfound3/new_pattern19.csv")
pattern_new20 = pd.read_csv("../input/newfound3/new_pattern20.csv")
pattern_new21 = pd.read_csv("../input/newfound3/new_pattern21.csv")
pattern_new22 = pd.read_csv("../input/newfound3/new_pattern22.csv")
pattern_new23 = pd.read_csv("../input/newfound3/new_pattern23.csv")
pattern_new24 = pd.read_csv("../input/newfound3/new_pattern24.csv")
pattern_new25 = pd.read_csv("../input/newfound3/new_pattern25.csv")
pattern_new27 = pd.read_csv("../input/newfound3/new_pattern27.csv")
pattern_new28 = pd.read_csv("../input/newfound3/new_pattern28.csv")
pattern_new29 = pd.read_csv("../input/newfound3/new_pattern29.csv")
pattern_new30 = pd.read_csv("../input/newfound3/new_pattern30.csv")
#pattern_new31 = pd.read_csv("../input/newfound3/new_pattern31.csv")
pattern_new32 = pd.read_csv("../input/newfound3/new_pattern32.csv")
pattern_new33 = pd.read_csv("../input/newfound3/new_pattern33.csv")
pattern_new34 = pd.read_csv("../input/newfound3/new_pattern34.csv")
pattern_new35 = pd.read_csv("../input/newfound3/new_pattern35.csv")
pattern_new36 = pd.read_csv("../input/newfound3/new_pattern36.csv")
pattern_new37 = pd.read_csv("../input/newfound3/new_pattern37.csv")
pattern_new38 = pd.read_csv("../input/newfound3/new_pattern38.csv")
pattern_new39 = pd.read_csv("../input/newfound3/new_pattern39.csv")

In [ ]:
pattern_1964666.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)
pattern_1166666.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)
pattern_812666.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)
pattern_2002166.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)
pattern_3160000.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)
pattern_3255483.drop(['Unnamed: 0','value_count'],axis=1,inplace=True)

In [ ]:
pattern_1166666.rename(columns={'8.50E+43': '850027e38'},inplace=True)

In [ ]:
l=[]
l.append(pattern_1964666.columns.values.tolist())
l.append(pattern_1166666.columns.values.tolist())
l.append(pattern_812666.columns.values.tolist())
l.append(pattern_2002166.columns.values.tolist())
l.append(pattern_3160000.columns.values.tolist())
l.append(pattern_3255483.columns.values.tolist())
l.append(pattern_new.columns.values.tolist())
l.append(pattern_new1.columns.values.tolist())
l.append(pattern_new2.columns.values.tolist())
l.append(pattern_new3.columns.values.tolist())
l.append(pattern_new4.columns.values.tolist())
l.append(pattern_new5.columns.values.tolist())
l.append(pattern_new7.columns.values.tolist())
l.append(pattern_new8.columns.values.tolist())
l.append(pattern_new9.columns.values.tolist())
l.append(pattern_new10.columns.values.tolist())
l.append(pattern_new11.columns.values.tolist())
l.append(pattern_new12.columns.values.tolist())
l.append(pattern_new13.columns.values.tolist())
l.append(pattern_new14.columns.values.tolist())
l.append(pattern_new15.columns.values.tolist())
l.append(pattern_new16.columns.values.tolist())
#l.append(pattern_new17.columns.values.tolist())
#l.append(pattern_new18.columns.values.tolist())
l.append(pattern_new19.columns.values.tolist())
#l.append(pattern_new20.columns.values.tolist())
#l.append(pattern_new21.columns.values.tolist())
#l.append(pattern_new22.columns.values.tolist())
#l.append(pattern_new23.columns.values.tolist())
#l.append(pattern_new24.columns.values.tolist())
#l.append(pattern_new25.columns.values.tolist())
#l.append(pattern_new26.columns.values.tolist())
#l.append(pattern_new27.columns.values.tolist())
#l.append(pattern_new28.columns.values.tolist())
#l.append(pattern_new29.columns.values.tolist())
#l.append(pattern_new30.columns.values.tolist())
#l.append(pattern_new32.columns.values.tolist())
#l.append(pattern_new33.columns.values.tolist())
#l.append(pattern_new34.columns.values.tolist())
#l.append(pattern_new35.columns.values.tolist())
#l.append(pattern_new36.columns.values.tolist())
#l.append(pattern_new37.columns.values.tolist())
#l.append(pattern_new38.columns.values.tolist())
#l.append(pattern_new39.columns.values.tolist())

In [ ]:
del l

In [ ]:
def _get_leak(df, cols,extra_feats, lag=0):
    f1 = cols[:((lag+2) * -1)]
    f2 = cols[(lag+2):]
    for ef in extra_feats:
        f1 += ef[:((lag+2) * -1)]
        f2 += ef[(lag+2):]
    
    d1 = df[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d1.to_csv('extra_d1.csv')
    d2 = df[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'}) 

    d2['pred'] = df[cols[lag]]
#     d2.to_csv('extra_d2.csv')
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d4 = d1[~d1.duplicated(['key'], keep=False)]
    d5 = d4.merge(d3, how='inner', on='key')
    
    d6 = d1.merge(d5, how='left', on='key')
    d6.to_csv('extra_d6.csv')
    
    return d1.merge(d5, how='left', on='key').pred.fillna(0)

In [ ]:
def compiled_leak_result():
    
    max_nlags = len(cols)-2
    train_leak = train[["ID", "target"] + cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        print('Processing lag', i)
        train_leak[c] = _get_leak(train, cols,l, i)
        
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts*1.0/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        print('Na count',tmp.compiled_leak.isna().sum())
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result


In [ ]:
train_leak, result = compiled_leak_result()